In [1]:
#import libraries

import numpy as np
from PIL import Image

In [28]:
#encoding function
def Encode(src, message, dest):

    img = Image.open(src, 'r')
    width, height = img.size
    array = np.array(list(img.getdata()))

    if img.mode == 'RGB':
        n = 3
        m = 0
    elif img.mode == 'RGBA':
        n = 4
        m = 1

    total_pixels = array.size//n

    message += "$t3g0"
    b_message = ''.join([format(ord(i), "08b") for i in message])
    req_pixels = len(b_message)

    if req_pixels > total_pixels:
        print("ERROR: Need larger file size")

    else:
        index=0
        for p in range(total_pixels):
            for q in range(m, n):
                if index < req_pixels:
                    array[p][q] = int(bin(array[p][q])[2:9] + b_message[index], 2)
                    index += 1

        array=array.reshape(height, width, n)
        enc_img = Image.fromarray(array.astype('uint8'), img.mode)
        enc_img.save(dest)
        print("Image Encoded Successfully")


#decoding function
def Decode(src):

    img = Image.open(src, 'r')
    array = np.array(list(img.getdata()))

    if img.mode == 'RGB':
        n = 3
        m = 0
    elif img.mode == 'RGBA':
        n = 4
        m = 1

    total_pixels = array.size//n

    hidden_bits = ""
    for p in range(total_pixels):
        for q in range(m, n):
            hidden_bits += (bin(array[p][q])[2:][-1])

    hidden_bits = [hidden_bits[i:i+8] for i in range(0, len(hidden_bits), 8)]

    message = ""
    for i in range(len(hidden_bits)):
        if message[-5:] == "$t3g0":
            break
        else:
            message += chr(int(hidden_bits[i], 2))
    if "$t3g0" in message:
        print("Hidden Message:", message[:-5])
    else:
        print("No Hidden Message Found")

#main function
def Stego():
    print("--Welcome to $t3g0--")
    print("1: Encode")
    print("2: Decode")

    func = input()

    if func == '1':
        print("Enter Source Image Path")
        src = input()
        print("Enter Message to Hide")
        message = input()
        print("Enter Destination Image Path")
        dest = input()
        print("Encoding...")
        Encode(src, message, dest)

    elif func == '2':
        print("Enter Source Image Path")
        src = input()
        print("Decoding...")
        Decode(src)

    else:
        print("ERROR: Invalid option chosen")

Stego()


--Welcome to $t3g0--
1: Encode
2: Decode
1
Enter Source Image Path
/content/lena2.png
Enter Message to Hide
the password is 123$$$
Enter Destination Image Path
/content/stegolena2.png
Encoding...
Image Encoded Successfully


In [34]:
from skimage import data, img_as_float
from skimage.metrics import structural_similarity as ssim
def ssimcal(img,stego):
  s=ssim(img,stego,multichannel= True)
  return s

In [35]:
def mse(original, stego):
	# the 'Mean Squared Error' between the two images is the
	# sum of the squared difference between the two images;
	# NOTE: the two images must have the same dimension
	err = np.sum((original.astype("float") - stego.astype("float")) ** 2)
	err /= float(original.shape[0] * stego.shape[1])

	# return the MSE, the lower the error, the more "similar"
	# the two images are
	return err

In [39]:
import cv2
from google.colab.patches import cv2_imshow
lenaoriginal_png = cv2.imread('/content/lena2.png')
lenastego_png=cv2.imread('/content/stegolena2.png')

lenassim_png=ssimcal(lenaoriginal_png,lenastego_png)
lenamse_png=mse(lenaoriginal_png,lenastego_png)
print("SSIM value for png image =",lenassim_png)
print("mse value for png  image =",lenamse_png)

original_jpeg = cv2.imread('/content/lena.jpeg')
stego_jpeg=cv2.imread('/content/stegolena.jpeg')

lenassim_jpeg=ssimcal(lenaoriginal_png,lenastego_png)
lenamse_jpeg=mse(lenaoriginal_png,lenastego_png)
print("SSIM value for jpeg  image =",lenassim_jpeg)
print("mse value for jpeg  image =",lenamse_jpeg)

#Lower the mse value more is the image similar
#Higher the ssim more is the image similar

SSIM value for png image = 0.9991834879138047
mse value for png  image = 23.69528888888889
SSIM value for jpeg  image = 0.9991834879138047
mse value for jpeg  image = 23.69528888888889


<ipython-input-34-4cf92be4d4aa>:4: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  s=ssim(img,stego,multichannel= True)
